In [56]:
# анализируем несколько .mol файлов
import numpy as np
import pandas as pd


def conn_type(comm):
    num_of_double = 0
    num_of_triple = 0
    for line in comm:
        if line[2] == '2':
            num_of_double += 1
        elif line[2] == '3':
            num_of_triple += 1
    if num_of_double == 0 and num_of_triple == 0:
        return 's'
    elif num_of_double == 1 and num_of_triple == 0:
        return 'd'
    elif num_of_double >= 2 and num_of_triple == 0:
        return 'w'
    elif num_of_triple > 0:
        return 't'
    else:
        print('WHAT')
        return '?'
    
def analize_mol_file1(molekula, file):# 1-цепочки
    with open(file) as f:
        lines = f.readlines()
        num_of_atoms = int(list(lines[3].split())[0])
        num_of_edges = int(list(lines[3].split())[1])
        end = lines.index('M  END\n')
        beg = 4 + num_of_atoms
        for elem in range(1, num_of_atoms+1):
            NN = lines[3+elem].split()[3] 
            num_of_neighbours = 0
            communic = [] #массив связей с конкретным элементом

            for i in range(end-beg):
                if int(list(lines[beg+i].split())[0]) == elem:
                    num_of_neighbours += 1
                    communic.append(list(lines[beg+i].split())[0:3])
                if int(list(lines[beg+i].split())[1]) == elem:
                    num_of_neighbours += 1
                    communic.append(list(lines[beg+i].split())[0:3])

            num_of_neighbours = str(num_of_neighbours)
            type_of_connection = conn_type(communic)
            name = NN + num_of_neighbours + type_of_connection

            if file in molekula.index:
                if name in molekula.columns:
                    if np.isnan(molekula[name][file]):
                        molekula.loc[file, name] = 1
                    else:
                        molekula.loc[file, name] += 1
                else:
                    molekula.loc[file, name] = 1
            else:
                molekula.loc[file, name] = 1
        

def analize_mol_file2(molekula, file):# 2-цепочки
    with open(file) as f:
        lines = f.readlines()
        num_of_atoms = int(list(lines[3].split())[0])
        num_of_edges = int(list(lines[3].split())[1])
        end = lines.index('M  END\n')
        beg = 4 + num_of_atoms
        for elem in range(1, num_of_atoms+1):
            connect = [] #массив связей с большим по номеру элементом

            for i in range(end-beg):
                if int(list(lines[beg+i].split())[0]) == elem and list(lines[beg+i].split())[1] > str(elem):
                    connect.append(list(lines[beg+i].split())[0:3])
                if int(list(lines[beg+i].split())[1]) == elem and list(lines[beg+i].split())[0] > str(elem):
                    connect.append(list(lines[beg+i].split())[0:3])
            for num in range(len(connect)):
                if int(connect[num][0]) == elem:
                    elem2 = int(connect[num][1])
                else:
                    elem2 = int(connect[num][0])
                NN1 = lines[3+elem].split()[3]
                NN2 = lines[3+elem2].split()[3]
                num_of_neighbours1 = 0
                num_of_neighbours2 = 0
                communic1 = []
                communic2 = []
                for i in range(end-beg):
                    if int(list(lines[beg+i].split())[0]) == elem:
                        num_of_neighbours1 += 1
                        communic1.append(list(lines[beg+i].split())[0:3])
                    if int(list(lines[beg+i].split())[1]) == elem:
                        num_of_neighbours1 += 1
                        communic1.append(list(lines[beg+i].split())[0:3])
                    if int(list(lines[beg+i].split())[0]) == elem2:
                        num_of_neighbours2 += 1
                        communic2.append(list(lines[beg+i].split())[0:3])
                    if int(list(lines[beg+i].split())[1]) == elem2:
                        num_of_neighbours2 += 1
                        communic2.append(list(lines[beg+i].split())[0:3])
                num_of_neighbours1 = str(num_of_neighbours1)
                type_of_connection1 = conn_type(communic1)
                num_of_neighbours2 = str(num_of_neighbours2)
                type_of_connection2 = conn_type(communic2)
                name1 = NN1 + num_of_neighbours1 + type_of_connection1
                name2 = NN2 + num_of_neighbours2 + type_of_connection2
                if name1 < name2:
                    name = name1 + name2
                else:
                    name = name2 + name1

                if file in molekula.index:
                    if name in molekula.columns:
                        if np.isnan(molekula[name][file]):
                            molekula.loc[file, name] = 1
                        else:
                            molekula.loc[file, name] += 1
                    else:
                        molekula.loc[file, name] = 1
                else:
                    molekula.loc[file, name] = 1
                

molekula = pd.DataFrame({})
# files = ["adrenalin.mol", "soil-1.mol"]
files = ["adrenalin.mol", "amaron.mol", "iervin.mol", 'acetona_peroxid.mol', 'aleiritic_acid.mol', "soil-1.mol"]
for file in files:
    analize_mol_file1(molekula, file)
molekula = molekula.fillna(0)
for file in files:
    analize_mol_file2(molekula, file)
molekula = molekula.fillna(0)
molekula

,C2d,C3d,C3s,C2s,N2s,C1s,O1s,N2d,C4s,O2s,...,C4sO2s,C1sC3d,C3sO2s,C1sC3s,C3sN2s,O2sO2s,C2sO1s,C3dN2s,C3dO2s,C1sO2s
adrenalin.mol,3.0,3.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amaron.mol,20.0,8.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iervin.mol,1.0,4.0,8.0,8.0,1.0,4.0,1.0,0.0,2.0,1.0,...,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
acetona_peroxid.mol,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,4.0,...,4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
aleiritic_acid.mol,0.0,1.0,2.0,13.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
soil-1.mol,4.0,3.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


In [121]:
#анализируем .sdf файл

import numpy as np
import pandas as pd


def conn_type_big(comm, atom, connections):
    num_of_double = 0
    num_of_triple = 0
    if connections == False:
        return '*'
    for row in comm:
        if row[0] == atom or row[1] == atom:
            if row[2] == 2:
                num_of_double += 1
            elif row[2] == 3:
                num_of_triple += 1
    if num_of_double == 0 and num_of_triple == 0:
        return 's'
    elif num_of_double == 1 and num_of_triple == 0:
        return 'd'
    elif num_of_double >= 2 and num_of_triple == 0:
        return 'w'
    elif num_of_triple > 0:
        return 't'
    else:
        print('WHAT')
        return '?'


def analize_sdf_file(mols, file, key, chain_num, neighbors=True, connections=True):
    with open(file) as f:
        lines = f.readlines()
        begin = []
        end = []
        for i, line in enumerate(lines):
            if line == key:
                begin.append(i)
            if line == 'M  END\n':
#             if line == '>  <ID>\n':
                end.append(i)
        for molecula in range(len(begin)):
            num_of_atoms = int(list(lines[begin[molecula]+2].split())[0])
            num_of_edges = int(list(lines[begin[molecula]+2].split())[1])
            beg = num_of_atoms + begin[molecula] + 3
            connect = []
            nums_to_names = dict()
            nums_to_names2 = dict()
            nums_to_names3 = dict()
            for i in range(end[molecula]-beg):
                connect.append(list(map(int, list(lines[beg+i].split())[0:3])))
            # 1-цепочки
            for atom in range(1, num_of_atoms+1):
                NN = lines[begin[molecula]+2+atom].split()[3]
                if len(NN) == 1:
                    NN = NN + '_'
                num_of_neighbors = 0
                for row in connect:
                    if atom == row[0] or atom == row[1]:
                        num_of_neighbors += 1
                num_of_neighbors = str(num_of_neighbors)
                type_of_connection = conn_type_big(connect, atom, connections)
                name = NN
                if neighbors == True:
                    name += num_of_neighbors
                else:
                    name += '*'
                name += type_of_connection + '*'
#                 name = NN + num_of_neighbors + type_of_connection + '*'
                nums_to_names[atom] = name
                if molecula in mols.index:
                    if name in mols.columns:
                        if np.isnan(mols[name][molecula]):
                            mols.loc[molecula, name] = 1
                        else:
                            mols.loc[molecula, name] += 1
                    else:
                        mols.loc[molecula, name] = 1
                else:
                    mols.loc[molecula, name] = 1
            # 2-цепочки
            if chain_num > 1:
                for atom in range(1, num_of_atoms+1):
                    for row in connect:
                        if (row[0] == atom and row[1] > atom) or (row[1] == atom and row[0] > atom):
                            NNmas = sorted([nums_to_names[row[0]], nums_to_names[row[1]]])
                            NN = NNmas[0] + NNmas[1]
                            name = NN
    #                         if row[2] == 1:
    #                             name = NN + '_s'
    #                         elif row[2] == 2:
    #                             name = NN + '_d'
    #                         elif row[2] == 3:
    #                             name == NN + '_t'
    #                         else:
    #                             name == NN + '_?'
                            nums_to_names2[tuple(sorted((row[0], row[1])))] = name
                            if molecula in mols.index:
                                if name in mols.columns:
                                    if np.isnan(mols[name][molecula]):
                                        mols.loc[molecula, name] = 1
                                    else:
                                        mols.loc[molecula, name] += 1
                                else:
                                    mols.loc[molecula, name] = 1
                            else:
                                mols.loc[molecula, name] = 1
            # 3-цепочки
            if chain_num > 2:
                for pair in nums_to_names2.keys():
                    for row in connect:
                        if (row[0] == pair[0] and row[1] > pair[1]) or (row[0] == pair[1] and row[1] > pair[0]):
                            if nums_to_names3.get(tuple(sorted([pair[0], pair[1], row[1]])), -1) == -1:                          
                                NNmas = sorted([nums_to_names2[pair], nums_to_names[row[1]]])
                                name = NNmas[0] + NNmas[1]
                                nums_to_names3[tuple(sorted([pair[0], pair[1], row[1]]))] = name
                        if (row[1] == pair[0] and row[0] > pair[1]) or (row[1] == pair[1] and row[0] > pair[0]):
                            if nums_to_names3.get(tuple(sorted([pair[0], pair[1], row[0]])), -1) == -1:                          
                                NNmas = sorted([nums_to_names2[pair], nums_to_names[row[0]]])
                                name = NNmas[0] + NNmas[1]
                                nums_to_names3[tuple(sorted([pair[0], pair[1], row[0]]))] = name
                        if molecula in mols.index:
                            if name in mols.columns:
                                if np.isnan(mols[name][molecula]):
                                    mols.loc[molecula, name] = 1
                                else:
                                    mols.loc[molecula, name] += 1
                            else:
                                mols.loc[molecula, name] = 1
                        else:
                            mols.loc[molecula, name] = 1

                
        

In [7]:
#добавим вектор y с признаком key
def find_y_sdf(file, y, key):
    with open(file) as f:
        lines = f.readlines()
        num = 0
        for i, line in enumerate(lines):
            if key in line:
                y.loc[num, key] = float(lines[i+1])
                num += 1
                
def find_y_set(file, y, key):
    with open(file) as f:
        lines = f.readlines()
        num = 0
        for i, line in enumerate(lines):
            if key in line:
                y.loc[num, key] = float(lines[i][2])
                num += 1

In [15]:
import pandas as pd

# y = pd.DataFrame({})
file = 'SOIL/soil-1-1.sdf'
key = '>  <logKow(cal)> '
find_y_sdf(file, y, key)
y = y.fillna(0)
y

,> <logKoc(exp)>,> <logKoc(cal)>,> <logKow(cal)>
0,1.40,1.60,1.18
1,1.43,1.56,1.10
2,1.95,2.12,1.99
3,1.94,1.98,1.64
4,1.57,1.64,1.30
...,...,...,...
323,2.16,2.55,3.51
324,1.98,2.13,2.14
325,1.87,1.81,1.69
326,1.80,1.75,1.57


In [16]:
f = 'SOIL/soil-1-1_y.csv'
y.to_csv(f, header=True, index=False, sep=',')



In [134]:
mols = pd.DataFrame({})
file = 'SOIL/soil-1-1.sdf'
key = '   - NASAWIN -\n'
analize_sdf_file(mols, file, key, 2, neighbors=True, connections=False)
mols = mols.fillna(0)
s = sorted(mols.columns)
mols = mols[sorted(s, key = lambda name: len(name))]
mols

,BR1**,CL1**,C_0**,C_1**,C_2**,C_3**,C_4**,F_1**,N_1**,N_2**,...,N_3**S_2**,N_3**S_4**,O_1**P_4**,O_1**S_3**,O_1**S_4**,O_2**P_4**,O_2**S_2**,O_2**S_4**,P_4**S_1**,P_4**S_2**
0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,5.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,4.0,3.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,4.0,3.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,4.0,3.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0.0,0.0,0.0,1.0,6.0,3.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
324,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
326,0.0,0.0,0.0,1.0,3.0,3.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
file = 'SOIL/soil-1-1.sdf'
with open(file) as f:
    i = 0
    for line in f.readlines():
        print(i, line)
        i+=1
        if i == 50:
            break
        if i == 37:
            h = line
            

0 

1 

2    - NASAWIN -

3 

4  12 12  0  0  0  0  0  0  0  0  1 V2000

5   141.0000  179.0000    0.0000 C   0  0  1  0  0  0

6   156.0000  205.0000    0.0000 C   0  0  1  0  0  0

7   186.0000  205.0000    0.0000 C   0  0  0  0  0  0

8   201.0000  179.0000    0.0000 C   0  0  1  0  0  0

9   186.0000  153.0000    0.0000 C   0  0  1  0  0  0

10   156.0000  153.0000    0.0000 C   0  0  0  0  0  0

11   149.0000  118.0000    0.0000 N   0  0  1  0  0  0

12   149.0000   88.0000    0.0000 C   0  0  0  0  0  0

13   149.0000   58.0000    0.0000 C   0  0  3  0  0  0

14   193.0000   88.0000    0.0000 O   0  0  0  0  0  0

15   201.0000  231.0000    0.0000 O   0  0  0  0  0  0

16   216.0000  257.0000    0.0000 C   0  0  3  0  0  0

17   1  2  2  0  0  0

18   1  6  1  0  0  0

19   2  3  1  0  0  0

20   3  4  2  0  0  0

21   3 11  1  0  0  0

22   4  5  1  0  0  0

23   5  6  2  0  0  0

24   6  7  1  0  0  0

25   7  8  1  0  0  0

26   8  9  1  0  0  0

27   8 10  2  0  0  0

28  11 

In [14]:
h

'>  <logKow(cal)> (1)\n'

In [135]:
# положить алфавит в csv

import csv

f = 'SOIL/soil-1-1_matrix_alphabet_NNd**_1and2chains.csv'
mols.to_csv(f, header=False, index=False)

with open('SOIL/soil-1-1_columns_alphabet_NNd**_1and2chains.csv', "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(list(mols.columns))
    
with open('SOIL/soil-1-1_rows_alphabet_NNd**_1and2chains.csv', "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(list(mols.index))

In [136]:
mols12 = pd.DataFrame({})
for col1 in mols.columns:
    for col2 in mols.columns:
        if col2 >= col1:
            mols12[col1+col2] = mols[col1] * mols[col2]
s = sorted(mols12.columns)
mols12 = mols12[sorted(s, key = lambda name: len(name))]

In [137]:
f = 'SOIL/soil-1-1_matrix_NNd**_1and2chains.csv'
mols12.to_csv(f, header=False, index=False)

with open('SOIL/soil-1-1_columns_NNd**_1and2chains.csv', "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(list(mols12.columns))
    
with open('SOIL/soil-1-1_rows_NNd**_1and2chains.csv', "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(list(mols12.index))

In [65]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

model = DecisionTreeRegressor(random_state=0)

In [66]:
X_train = mols[:250]
X_test = mols[250:]
y_train = y[:250]
y_test = y[250:]

In [67]:
cross_val_score(model, X_train, y_train, cv=5)

array([-0.35468029, -0.77783154, -0.10787061, -0.18852403,  0.24004299])

In [69]:
model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [74]:
y_pred = model.predict(X_test)

In [77]:
model.score(X_test, y_test)

-0.9221567571478027

In [79]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.plot(X_test, model.predict(X_test), color ='blue')

In [80]:
# from __future__ import print_function
# from rdkit import Chem

mol_pairs

In [156]:
#Матрица расстояний для двух молекул

with open(file) as f:
    lines = f.readlines()
    begin = []
    end = []
    for i, line in enumerate(lines):
        if line == key:
            begin.append(i)
        if line == 'M  END\n':
            end.append(i)
    num_of_atoms = int(list(lines[begin[0]+2].split())[0])
    beg = num_of_atoms + begin[0] + 3
    matrix = np.zeros(num_of_atoms*num_of_atoms).reshape(num_of_atoms, num_of_atoms)
    connect = []
    for i in range(end[0]-beg):
        connect.append(list(map(int, list(lines[beg+i].split())[0:3])))
    for row in connect:
        matrix[row[0]-1][row[1]-1] = 1
        matrix[row[1]-1][row[0]-1] = 1
    for i in range(num_of_atoms):
        matrix[i][i] = 1
print(matrix)
distanceMatrix(matrix)

[[1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


https://scikit-learn.org/0.20/modules/generated/sklearn.cluster.DBSCAN.html